# Your First Binary Simulations with POSYDON 🌠

**Tutorial goal:**

In this tutorial, we will run a small population of 10 binaries locally, and explore how to manipulate the output data from your population.

**New concepts:**

- Population ini file

If you haven't done so yet, export the path POSYDON environment variables.
Set these parameters in your `.bash_profile` or `.zshrc` if you use POSYDON regularly.

In [ ]:
%env PATH_TO_POSYDON=/YOUR/POSYDON/PATH/
%env PATH_TO_POSYDON_DATA=/YOUR/POSYDON_DATA/PATH/

## Creating the Initialisation File

----
To run population synthesis with POSYDON, a `population_params.ini` file is required.
This file described how the stellar population is created and what prescriptions and parameters are implemented in specific steps.

POSYDON comes with a default `population_params_default.ini` file found at `PATH_TO_POSYDON/posydon/popsyn` or have a look [here](population_params.ini).

The file is split in three main parts. You can find more details about their properties by clicking on their links.
1. **[SimulationProperties]():**
   - these describe the properties and parameters of different steps in the evolution of a binary systems.
2. **[BinaryPopulation]():** 
   - parameters of the initial sampling of the binary population, such as initial mass function, period distribution, and metallicity.
   - Also contains parameters on how the population is ran, such as how many binaries are kept in memory.
3. **[SavingOutput]()**
   - Describes the data from the binary and each individual star to the output files.

We will copy the default population run parameter file to the current folder.

In [1]:
import os
import shutil
from posydon.config import PATH_TO_POSYDON

path_to_params = os.path.join(PATH_TO_POSYDON, "posydon/popsyn/population_params_default.ini")
shutil.copyfile(path_to_params, './population_params.ini')

'./population_params.ini'


<div class="alert alert-block alert-warning"><b>Reprocessed POSYDON v1 data</b> 

If you're using the reprocessed POSYDON v1 dataset, you will only have solar metallicity available!
This means you will only be able to run populations with `metallicity = [1]`!

You should be able to follow along with this tutorial and can follow along with the "One metallicity notebook", but the multi-metallicity component is not yet available.
<div>

# Creating and running a binary population

The copied `population_params.ini` contains the parameters to run 10 binaries at a metallicity of $Z=1 Z_\odot$.

If you open the file and scroll down to the **BinaryPopulation** section, you will see how they're defined:

```
metallicity = [1] # [2., 1., 0.45, 0.2, 0.1, 0.01, 0.001, 0.0001]
# In units of solar metallicity
...
number_of_binaries = 10
# int
```

If you like to run a small population in a notebook, you can use the `PopulationRunner` to do this. If you want to run a specific binary instead, have a look at the [Binary Tutorial]().

The `PopulationRunner` class takes the `.ini` file and sets-up a population run.

This will create [BinaryPopulations]() for each metallicity defined in the `.ini` file.
In this case, we can check and see that a single BinaryPopulation is created and contains 10 binaries.

In [2]:
from posydon.popsyn.synthetic_population import PopulationRunner
poprun = PopulationRunner('./population_params.ini', verbose=True)

In [3]:
print('Number of binary populations:',len(poprun.binary_populations))
print('Metallicity:', poprun.binary_populations[0].metallicity)
print('Number of binaries:', poprun.binary_populations[0].number_of_binaries)

Number of binary populations: 1
Metallicity: 1
Number of binaries: 10


For this tutorial, we set `verbose=True`, which shows you the progress of the population run.
This overwrites the population verbose set inside the `population_params.ini` file.

Now we are ready to evolve the binary population. This should take about 30 seconds, but depends on your machine.

In [4]:
poprun.evolve()

100%|██████████| 10/10 [00:10<00:00,  1.05s/it]


Merging 1 files...
Files merged!
Removing files in /Users/max/Documents/POSYDON/docs/_source/tutorials-examples/population-synthesis/1e+00_Zsun_batches...


# Inspecting the population: Population class

When you ran the population, you might have seen that a temporary folder with the name `1e+00_Zsun_batches` was created while the binaries were evolved.
This is a temporary folder in which populations are temporarly saved.
After the binary evolution has finished, the binaries in the folder are moved to a single file named `1e+00_Zsun_popululation.h5`. This is done automatically, when you run a population using the `PopulationRunner` class.

The created file contains 3 main components:

1. **history:** the evolution of an individual binary in a pandas DataFrame
2. **oneline:** a single line to describe the initial and final conditions and some one-of parameters, such as the metallicity.
3. **mass_per_metallicity:** some metadata on the population, such as the total simulated mass, the actual underlying mass of the population, and the number of binaries in the file.

The `Population` provides an interface with these components in the file, such that you're able to share the populations runs and can work with large population that do not fit in memory.


<div class="alert alert-block alert-warning"><b>Older Population Files</b> 

If you're using older population files, you can make them compatible with the `Population` class by calling `Population(pop_file, metallicity,ini_file)`, where the `metallicity` is in solar units. You will only need to do this once; afterwards you can initialise the class like normal.<div>

In [1]:
from posydon.popsyn.synthetic_population import Population

In [2]:
pop = Population('1e+00_Zsun_population.h5', verbose=True)

mass_per_metallicity table read from population file!


/Users/max/Documents/POSYDON/posydon/popsyn/synthetic_population.py:996: UserWarning: 1e+00_Zsun_population.h5 does not contain formation channels!
  warnings.warn(f"{filename} does not contain formation channels!")


The `pop.mas_per_met` shows some basic information about the population you've just created.


1. The index (**metallicity**) is the metallicity of your population in solar units.
2. **simulated mass** is the total ZAMS mass that has been evolved in the population.
3. **underlying mass** is the actual mass of the population if one integrates the IMF and period distribution fully.
4. **number_of_systems** shows the 10 systems in the file.

In [7]:
pop.mass_per_metallicity

,simulated_mass,underlying_mass,number_of_systems
metallicity,,,
1,212.590742,1195.573402,10


There are some additional metadata properties available, such as:
- `metallicities`: the metallicity in absolute metallicity
- `solar_metallicities`: the metallicities in the file in solar metallicity
- `number_of_systems`: the total number of systems in the Population file
- `indices`: the indices of the binaries in the file
- `columns`: the columns available in the `history` and `oneline` dataframes

In [8]:
# you can also access the total number of systems in the file with
print(pop.number_of_systems)

10


## Population.history

In [9]:
pop.history.lengths

,length
index,
0,8
1,6
2,6
3,6
4,6
5,6
6,6
7,6
8,10


`pop.history` loads in the full history of all the binaries into memory.
You can access individual or a selections of the population using several methods:


1. pop.history[5]
2. pop.history[[0,4]]
3. pop.history['time]
4. pop.history.select()

The `select` function is the most powerfull way to access the binaries, because it allows you to perform selections based on the specific columns available in the history dataframe.
For example, below we can select on `state == 'RLO1'`, which gives us all the rows with RLO1 occuring.

The columns available are limited to string columns: `state`, `event`, `step_names`, `S1_state`, `S2_state` and index and columns names.

In [49]:
# select only binary_index 5
pop.history[5]

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
5,detached,ZAMS,0.000000e+00,18.030445,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,12.674669,...,NaN,NaN,NaN,NaN,0.715500,2.703000e-01,NaN,NaN,NaN,NaN
5,detached,CC1,1.924170e+07,19.940187,0.0,-99.0,step_HMS_HMS,0.011357,stripped_He_Central_C_depletion,4.086053,...,0.00000,0.0,0.000000,0.000000,0.544525,4.415872e-01,0.713544,0.272169,0.758103,14.810582
5,disrupted,NaN,1.924170e+07,NaN,NaN,NaN,step_SN,0.000336,NS,1.334688,...,0.00000,0.0,0.000000,0.000000,0.544525,4.415872e-01,0.713544,0.272169,0.758103,14.810582
5,disrupted,CC2,4.586978e+07,NaN,NaN,NaN,step_disrupted,0.226731,NS,1.334688,...,2.78991,NaN,1.733408,0.043991,0.000000,4.552803e-14,0.689894,0.295821,0.011219,8.550198
5,disrupted,NaN,4.586978e+07,NaN,NaN,NaN,step_SN,0.000453,NS,1.334688,...,NaN,NaN,1.733408,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
5,disrupted,END,4.586978e+07,NaN,NaN,NaN,step_end,0.000022,NS,1.334688,...,NaN,NaN,1.733408,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [11]:
# select binary 0 and 4
pop.history[[0,4]]

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
0,detached,ZAMS,0.000000e+00,12.550069,0.000000,NaN,initial_cond,0.000000,H-rich_Core_H_burning,7.756697,...,NaN,NaN,NaN,NaN,0.715500,2.703000e-01,NaN,NaN,NaN,NaN
0,detached,CC1,4.772680e+07,2.837546,0.000000,-60.652948,step_HMS_HMS,0.110728,stripped_He_Central_He_depleted,1.426284,...,1.976869e-15,5.353742e-16,9.896928e-16,2.940361e-17,0.657017,3.288611e-01,0.650228,0.335574,0.883802,17.020685
0,detached,NaN,4.772680e+07,2.486149,0.132085,NaN,step_SN,0.000347,NS,1.260782,...,1.976869e-15,5.353742e-16,9.896928e-16,2.940361e-17,0.657017,3.288611e-01,0.650228,0.335574,0.883802,17.020685
0,RLO2,oRLO2,2.605121e+08,2.231762,0.000000,NaN,step_detached,2.123813,NS,1.260782,...,3.324989e-01,NaN,0.000000e+00,0.000000e+00,0.000002,9.861392e-01,0.715500,0.270271,0.292356,6.286774
0,RLO2,oCE2,3.224839e+08,0.932548,0.000000,-2.084313,step_CO_HMS_RLO,0.116093,NS,1.289076,...,5.941028e-02,1.395877e-02,0.000000e+00,0.000000e+00,0.085216,9.008807e-01,0.706444,0.279250,0.024500,2.249569
0,merged,oMerging2,3.224839e+08,0.932548,0.000000,-2.084313,step_CE,0.000030,NS,1.289076,...,5.941028e-02,1.395877e-02,0.000000e+00,0.000000e+00,0.085216,9.008807e-01,0.706444,0.279250,0.024500,2.249569
0,merged,NaN,3.224839e+08,NaN,NaN,-2.084313,step_merged,0.000145,NS,1.289076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,merged,END,3.224839e+08,NaN,NaN,-2.084313,step_end,0.000016,NS,1.289076,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,detached,ZAMS,0.000000e+00,63.841765,0.000000,NaN,initial_cond,0.000000,H-rich_Core_H_burning,20.685378,...,NaN,NaN,NaN,NaN,0.715500,2.703000e-01,NaN,NaN,NaN,NaN


In [12]:
pop.history['time']

,time
binary_index,
0,0.000000e+00
0,4.772680e+07
0,4.772680e+07
0,2.605121e+08
0,3.224839e+08
...,...
9,9.746649e+06
9,9.746649e+06
9,1.238097e+07


You can also check what columns are available in the history file:

In [3]:
pop.history.columns

['state',
 'event',
 'time',
 'orbital_period',
 'eccentricity',
 'lg_mtransfer_rate',
 'step_names',
 'step_times',
 'S1_state',
 'S1_mass',
 'S1_log_R',
 'S1_log_L',
 'S1_lg_mdot',
 'S1_he_core_mass',
 'S1_he_core_radius',
 'S1_co_core_mass',
 'S1_co_core_radius',
 'S1_center_h1',
 'S1_center_he4',
 'S1_surface_h1',
 'S1_surface_he4',
 'S1_surf_avg_omega_div_omega_crit',
 'S1_spin',
 'S2_state',
 'S2_mass',
 'S2_log_R',
 'S2_log_L',
 'S2_lg_mdot',
 'S2_he_core_mass',
 'S2_he_core_radius',
 'S2_co_core_mass',
 'S2_co_core_radius',
 'S2_center_h1',
 'S2_center_he4',
 'S2_surface_h1',
 'S2_surface_he4',
 'S2_surf_avg_omega_div_omega_crit',
 'S2_spin']

In [39]:
# using the select function
pop.history.select(where='index == 9')

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
9,detached,ZAMS,0.000000e+00,1.438075,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,17.864323,...,NaN,NaN,NaN,NaN,0.715500,0.270300,NaN,NaN,NaN,NaN
9,contact,oCE1,9.746649e+06,1.555783,0.0,-5.010713,step_HMS_HMS,0.009001,H-rich_Core_H_burning,11.784814,...,0.0,0.0,0.0,0.0,0.360889,0.625233,0.712245,0.273428,0.594361,8.975468
9,merged,oMerging1,9.746649e+06,1.555783,0.0,-5.010713,step_CE,0.000021,H-rich_Core_H_burning,11.784814,...,0.0,0.0,0.0,0.0,0.360889,0.625233,0.712245,0.273428,0.594361,8.975468
9,merged,CC1,1.238097e+07,NaN,NaN,NaN,step_merged,0.236094,stripped_He_Central_C_depletion,13.080039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,merged,NaN,1.238097e+07,NaN,NaN,NaN,step_SN,0.038439,BH,12.580039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,merged,END,1.238097e+07,NaN,NaN,NaN,step_end,0.000017,BH,12.580039,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# selecting all RLO1 states and only time and state columns
pop.history.select(where='state == RLO2', columns=['time', 'state'])

,time,state
binary_index,,
0,2.605121e+08,RLO2
0,3.224839e+08,RLO2
8,7.037700e+07,RLO2
8,7.037703e+07,RLO2


In [41]:
# selecting rows 10 to 16
pop.history.select(start=10, stop=16)

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
1,merged,oMerging1,6.690491e+06,1.005247,0.0,-2.536244,step_CE,0.000022,H-rich_Core_H_burning,13.986610,...,0.0,0.0,0.0,0.0,0.609374,0.376706,0.714943,0.270869,0.989596,14.950735
1,merged,CC1,1.059788e+07,NaN,NaN,NaN,step_merged,2.262348,H-rich_Central_C_depletion,11.107115,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,merged,NaN,1.059788e+07,NaN,NaN,NaN,step_SN,0.033581,BH,10.406373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,merged,END,1.059788e+07,NaN,NaN,NaN,step_end,0.000019,BH,10.406373,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,detached,ZAMS,0.000000e+00,268.458519,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,12.013690,...,NaN,NaN,NaN,NaN,0.715500,0.270300,NaN,NaN,NaN,NaN
2,detached,CC1,2.097134e+07,957.804955,0.0,-99.000000,step_HMS_HMS,0.538816,stripped_He_Central_C_depletion,4.076633,...,0.0,0.0,0.0,0.0,0.047427,0.938716,0.712787,0.272987,0.533220,6.953203


You might have notices while using the above functions that not all the binaries will have the same length in the history.
You can access these with `pop.history_lengths`. This information is also stored in the population file.

In [42]:
pop.history_lengths

,length
index,
0,8
1,6
2,6
3,6
4,6
5,6
6,6
7,6
8,10


## Population.oneline

`Population.oneline` provides a similar interface to accessing the DataFrame in the population file as `Population.history`, with similar functionality being available.

The `select` function only has acces to:
- `index`
- column names 
- string columns: `state_i`, `state_f`, `event_i`, `event_f`, `step_names_i`, `step_names_f`, `S1_state_i`, `S1_state_f`, `S2_state_i`, `S2_state_f`, `S1_SN_type`, `S2_SN_type`, `interp_class_HMS_HMS`, `interp_class_CO_HeMS`, `interp_class_CO_HMS_RLO`, `interp_class_CO_HeMS_RLO`, `mt_history_HMS_HMS`, `mt_history_CO_HeMS`, `mt_history_CO_HMS_RLO`, `mt_history_CO_HeMS_RLO`

In [45]:
pop.oneline[5]
pop.oneline[[0,4]]
pop.oneline.select(where='index == 9')
pop.oneline.select(where='index == [0,9]')

,state_i,event_i,time_i,orbital_period_i,eccentricity_i,lg_mtransfer_rate_i,step_names_i,step_times_i,state_f,event_f,...,interp_class_CO_HMS_RLO,interp_class_CO_HeMS,interp_class_CO_HeMS_RLO,mt_history_HMS_HMS,mt_history_CO_HMS_RLO,mt_history_CO_HeMS,mt_history_CO_HeMS_RLO,FAILED,WARNING,metallicity
binary_index,,,,,,,,,,,,,,,,,,,,,
0,detached,ZAMS,0.0,12.550069,0.0,NaN,initial_cond,0.0,merged,END,...,unstable_MT,NaN,NaN,Stable RLOF during stripped He star,Unstable RLOF during MS,NaN,NaN,0,1,1
9,detached,ZAMS,0.0,1.438075,0.0,NaN,initial_cond,0.0,merged,END,...,NaN,NaN,NaN,Unstable contact phase,NaN,NaN,NaN,0,1,1


In [46]:
pop.oneline.columns

Index(['state_i', 'event_i', 'time_i', 'orbital_period_i', 'eccentricity_i',
       'lg_mtransfer_rate_i', 'step_names_i', 'step_times_i', 'state_f',
       'event_f',
       ...
       'interp_class_CO_HMS_RLO', 'interp_class_CO_HeMS',
       'interp_class_CO_HeMS_RLO', 'mt_history_HMS_HMS',
       'mt_history_CO_HMS_RLO', 'mt_history_CO_HeMS', 'mt_history_CO_HeMS_RLO',
       'FAILED', 'WARNING', 'metallicity'],
      dtype='object', length=103)

## Population.formation_channels

You might be interested in figuring out what sort of formation pathways/channels a binary has followed through its evolution.

This is not a standard output of the population synthesis, but you can include it into the population file by calculating it. 
If you would like more detail on the initial mass transfer, you can set `mt_history=True`.

This will write the formation channels to the Population file, which can be accessed by `Population.formation_channels`.


In [18]:
pop.calculate_formation_channels(mt_history=True)

In [19]:
# the formation channels are loaded in with pop.formation_channels
pop.formation_channels

,channel_debug,channel
0,ZAMS_oRLO1_CC1_oRLO2_CC2_CO_contact_END,ZAMS_oRLO1_CC1_oRLO2_CC2_END
1,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
2,ZAMS_oRLO1_CC1_oRLO2_CC2_END,ZAMS_oRLO1_CC1_oRLO2_CC2_END
3,ZAMS_CC1_CC2_END,ZAMS_CC1_CC2_END
4,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
5,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
6,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
7,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
8,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
9,ZAMS_oRLO1_CC1_oRLO2_oCE2_oMerging2_END,ZAMS_oRLO1_CC1_oRLO2_oCE2_oMerging2_END


Next time you open this population file, the `formation_channels` will be available without having to be recalculated.

In [20]:
pop = Population('1e+00_Zsun_population.h5')
pop.formation_channels

,channel_debug,channel
0,ZAMS_oRLO1_CC1_oRLO2_CC2_CO_contact_END,ZAMS_oRLO1_CC1_oRLO2_CC2_END
1,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
2,ZAMS_oRLO1_CC1_oRLO2_CC2_END,ZAMS_oRLO1_CC1_oRLO2_CC2_END
3,ZAMS_CC1_CC2_END,ZAMS_CC1_CC2_END
4,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
5,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
6,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
7,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
8,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
9,ZAMS_oRLO1_CC1_oRLO2_oCE2_oMerging2_END,ZAMS_oRLO1_CC1_oRLO2_oCE2_oMerging2_END


## Selecting a sub-population

You might just want a small sub-selection of the full population, especially if you're working with large population and multi-metallicity runs.

The `Population.export_selection()` function will export just the indices of the binaries you're interested in into a new file.
The simulated and underlying mass will remain the same, since they are dependent on the population run.

If we select just 2 binaries and export them, we create a new population of just the binaries you're interested in.
In the [BBH analysis]() and [GRB analysis]() tutorials, we show how to perform a selection with multiple criteria and metallicities.



In [1]:
indices = [0,9]
pop.export_selection(indices, 'selected.h5')    

NameError: name 'pop' is not defined

In [24]:
selected = Population('selected.h5')
selected.mass_per_metallicity

,simulated_mass,underlying_mass,number_of_systems
0.0001,296.055678,1664.965703,2


If you would like to know the simulated mass of just your population, you can calulate this using the online ZAMS values.

In [ ]:
import numpy as np

mass_selection = np.sum(selected.oneline[['S1_mass_i', 'S2_mass_i']])

By default this export-selection will not overwrite nor append if the output file is already present.
You have to explicitly state what you would like to append to or overwrite the population file.


With the `append=True` you are able to combine multiple stellar populations into a single file. 
This is especially useful when creating multi-metallicity populations.

In [26]:
# this will overwrite the existing file
pop.export_selection(indices, 'selected.h5', overwrite=True)

selected = Population('selected.h5')
selected.mass_per_metallicity


,simulated_mass,underlying_mass,number_of_systems
0.0001,296.055678,1664.965703,2


In [27]:
# This will add to the file and add the extra simulated mass
pop.export_selection(indices, 'selected.h5', append=True)
selected = Population('selected.h5')
selected.mass_per_metallicity

,simulated_mass,underlying_mass,number_of_systems
0.0001,592.111356,3329.931407,4


<br/><br/>

Feel free to explore the small binary population you've just created!

If you want to learn more about population synthesis and how to build more complex models it is advised to continue with the remaining tutorials and consult the POSYDON documentation.

### Local MPI runs

To speed up population synthesis runs, you can run on a computing cluster, as described in [HPC Facilities](pop_syn), or you can distribute the population synthesis across multiple cores on your local machine using MPI.

To enable local MPI runs, go into the `population_params.ini`  and change `use_MPI` to `True`.

It's important to note that you cannot run have this option enabled for cluster runs!

We create a binary population simulation script to run the population:

In [ ]:
%%writefile script.py
from posydon.popsyn.synthetic_population import PopulationRunner

if __name__ == "__main__":
    synth_pop = PopulationRunner("./population_params.ini")
    synth_pop.evolve()

This script can be initiated using a local where `NR_processors` is the number of processors you would like to us.

In [ ]:
mpiexec -n ${NR_processors} python script.py

This will create a folder for each metallicity in the population and store output of the parallel runs in it.

You will have to concatenate these runs manually into a single population file per metallicity, which can be achieved using the following code:

In [ ]:
from posydon.popsyn.synthetic_population import PopulationRunner

synth_pop = PopulationRunner("./population_params.ini")
for pop in synth_pop.binary_populations:
    synth_pop.merge_parallel_runs(pop)